<center><font color="green" size="6"> <b>Sugarcane Yield Prediction using RS and ML<b> </font></center>
<center><img src="https://www.omex.com/wp-content/uploads/2020/10/Sugar-Cane-Harvest-1536x772.jpg" height="150"></center>

---

_Script elaborated by **Raul Roberto Poppiel**_ ([raulpoppiel@gmail.com](raulpoppiel@gmail.com)) for the [FAPESP](https://fapesp.br/en) project Nº [23/01062-1](https://bv.fapesp.br/en/bolsas/207973/satellite-imagery-and-machine-learning-for-sugarcane-yield-estimation-in-regions-of-sao-paulo-state/), led by Professor Ana Claudia dos Santos Luciano (analuciano@usp.br) from ESALQ/USP, Brazil. The methodology employed was developed by Rafaella Pironato Amaro ([rafaellapironato.amaro@gmail.com](rafaellapironato.amaro@gmail.com)) and is detailed in the document titled [Estimativa de produtividade da cana-de-açúcar a partir de imagens do satélite Sentinel-2A e o algoritmo de aprendizagem de máquina Random Forest](https://doi.org/10.11606/D.11.2023.tde-02102023-163947).

### Install and import tools

Data science modules

In [ ]:
# Load modules
import os
import pandas as pd
from pathlib import Path

print('Modules loaded')

Modules loaded


### Connect to GDrive

In [ ]:
# Connect to Google Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### Set GDrive paths

Define the output folders in GD

In [ ]:
# Define folder names
folder_name_root = 'Colab Notebooks'
folder_name_project = '01_SugarcaneYieldPrediction'
folder_name_farm = 'usinas_all'
folder_name_specific = '08_merged_safra_all'

In [ ]:
# Check if the folder exists or else create
root_path = f'/content/drive/MyDrive/{folder_name_root}'
project_path = f'{root_path}/{folder_name_project}'
farm_path = f'{project_path}/{folder_name_farm}'
out_path = f'{farm_path}/{folder_name_specific}' # your results will be stored in 'out_path'

if not os.path.exists(out_path):
  Path(out_path).mkdir(parents=True, exist_ok=True)
  print("Output directory created successfully.")
else:
  print("Output directory already exists.")

os.chdir(out_path)
print(os.getcwd(),'\n')  # Print the current working directory
pd.DataFrame(os.listdir(), columns=['List files'])  # List files and directories in the current directory

Output directory already exists.
/content/drive/MyDrive/Colab Notebooks/01_SugarcaneYieldPrediction/02_usinas_all/08_merged_safra_all 

List of files:
                                       Files
0  08_all_safra_without_outliers_merged.csv


# Import data

In [ ]:
# Specify the folder path
folder_path = os.path.join(farm_path, '07_outliers_analysis')
folder_path

'/content/drive/MyDrive/Colab Notebooks/01_SugarcaneYieldPrediction/02_usinas_all/07_outliers_analysis'

In [ ]:
# Get a list of all files in the folder
all_files = os.listdir(folder_path)

# Filter files containing a pattern
pattern =  'without_outliers'

filtered_files = [file for file in all_files if pattern in file]
# filtered_files = filtered_files[:1]
filtered_files

['07_all_data_monthly_without_outliers_safra_1920.csv',
 '07_all_data_monthly_without_outliers_safra_2021.csv',
 '07_all_data_monthly_without_outliers_safra_2122.csv',
 '07_all_data_monthly_without_outliers_safra_2223.csv']

# Merge data

In [ ]:
# Initialize an empty DataFrame to store merged data
merged_data = pd.DataFrame()

# Loop through filtered files and merge them
for file in filtered_files:
    file_path = os.path.join(folder_path, file)
    current_data = pd.read_csv(file_path)

    # Check for duplicated headers and modify them for uniqueness
    seen_headers = {}
    modified_headers = []

    for header in current_data.columns:
        original_header = header[4:] if str(header[:4]).isdigit() else header
        count = seen_headers.get(original_header, 0)

        # Ensure uniqueness of headers
        if count > 0:
            modified_header = f"{str(int(original_header[:2]) + 12)}{original_header[2:]}"
        else:
            modified_header = original_header

        modified_headers.append(modified_header)
        seen_headers[original_header] = count + 1

    # Rename columns with modified headers
    current_data.columns = modified_headers

    # Assuming that the columns in all CSVs are the same, you can use concat
    merged_data = pd.concat([merged_data, current_data], ignore_index=True)

# Display the modified and merged DataFrame
pd.set_option('display.max_columns', None)  # Display all columns
merged_data

,ID_SIG,Local,AREA,COD_USINA,CODFAZ,BLOCO,TALHAO,VARIEDADE,TCH_ANT,POL,TAH,Unidade_So,Ordem,relevo,EST_Corte,VAR,SAFRA_real,soloGeral,SAFRA,usina,12_temperature_2m,12_total_precipitation_sum,12_surface_solar_radiation_downwards_sum,01_temperature_2m,01_total_precipitation_sum,01_surface_solar_radiation_downwards_sum,02_temperature_2m,02_total_precipitation_sum,02_surface_solar_radiation_downwards_sum,03_temperature_2m,03_total_precipitation_sum,03_surface_solar_radiation_downwards_sum,01_ALT,01_ARMf,01_ARMi,01_Def,01_Def_rel,01_ETP,01_ETR,01_ETc,01_Exc,01_Kc,01_NAC,01_P_ETc,02_ALT,02_ARMf,02_ARMi,02_Def,02_Def_rel,02_ETP,02_ETR,02_ETc,02_Exc,02_Kc,02_NAC,02_P_ETc,03_ALT,03_ARMf,03_ARMi,03_Def,03_Def_rel,03_ETP,03_ETR,03_ETc,03_Exc,03_Kc,03_NAC,03_P_ETc,04_ALT,04_ARMf,04_ARMi,04_Def,04_Def_rel,04_ETP,04_ETR,04_ETc,04_Exc,04_Kc,04_NAC,04_P_ETc,05_ALT,05_ARMf,05_ARMi,05_Def,05_Def_rel,05_ETP,05_ETR,05_ETc,05_Exc,05_Kc,05_NAC,05_P_ETc,06_ALT,06_ARMf,06_ARMi,06_Def,06_Def_rel,06_ETP,06_ETR,06_ETc,06_Exc,06_Kc,06_NAC,06_P_ETc,07_ALT,07_ARMf,07_ARMi,07_Def,07_Def_rel,07_ETP,07_ETR,07_ETc,07_Exc,07_Kc,07_NAC,07_P_ETc,08_ALT,08_ARMf,08_ARMi,08_Def,08_Def_rel,08_ETP,08_ETR,08_ETc,08_Exc,08_Kc,08_NAC,08_P_ETc,09_ALT,09_ARMf,09_ARMi,09_Def,09_Def_rel,09_ETP,09_ETR,09_ETc,09_Exc,09_Kc,09_NAC,09_P_ETc,10_ALT,10_ARMf,10_ARMi,10_Def,10_Def_rel,10_ETP,10_ETR,10_ETc,10_Exc,10_Kc,10_NAC,10_P_ETc,11_ALT,11_ARMf,11_ARMi,11_Def,11_Def_rel,11_ETP,11_ETR,11_ETc,11_Exc,11_Kc,11_NAC,11_P_ETc,12_ALT,12_ARMf,12_ARMi,12_Def,12_Def_rel,12_ETP,12_ETR,12_ETc,12_Exc,12_Kc,12_NAC,12_P_ETc,13_ALT,13_ARMf,13_ARMi,13_Def,13_Def_rel,13_ETP,13_ETR,13_ETc,13_Exc,13_Kc,13_NAC,13_P_ETc,14_ALT,14_ARMf,14_ARMi,14_Def,14_Def_rel,14_ETP,14_ETR,14_ETc,14_Exc,14_Kc,14_NAC,14_P_ETc,15_ALT,15_ARMf,15_ARMi,15_Def,15_Def_rel,15_ETP,15_ETR,15_ETc,15_Exc,15_Kc,15_NAC,15_P_ETc,12_NDVI,12_EVI,12_SAVI,12_NDMI,12_NDWI1,12_NDWI2,12_NDVIre1,12_NDVIre2,12_NDVIre3,12_CIre1,12_CIre2,12_CIre3,12_MSRre1,12_MSRre2,12_MSRre3,12_SIWSI,12_red,12_nir,12_swir1,12_swir2,12_rededge1,12_rededge2,12_rededge3,12_nir8A,01_NDVI,01_EVI,01_SAVI,01_NDMI,01_NDWI1,01_NDWI2,01_NDVIre1,01_NDVIre2,01_NDVIre3,01_CIre1,01_CIre2,01_CIre3,01_MSRre1,01_MSRre2,01_MSRre3,01_SIWSI,01_red,01_nir,01_swir1,01_swir2,01_rededge1,01_rededge2,01_rededge3,01_nir8A,02_NDVI,02_EVI,02_SAVI,02_NDMI,02_NDWI1,02_NDWI2,02_NDVIre1,02_NDVIre2,02_NDVIre3,02_CIre1,02_CIre2,02_CIre3,02_MSRre1,02_MSRre2,02_MSRre3,02_SIWSI,02_red,02_nir,02_swir1,02_swir2,02_rededge1,02_rededge2,02_rededge3,02_nir8A,03_NDVI,03_EVI,03_SAVI,03_NDMI,03_NDWI1,03_NDWI2,03_NDVIre1,03_NDVIre2,03_NDVIre3,03_CIre1,03_CIre2,03_CIre3,03_MSRre1,03_MSRre2,03_MSRre3,03_SIWSI,03_red,03_nir,03_swir1,03_swir2,03_rededge1,03_rededge2,03_rededge3,03_nir8A
0,X19200280001000100000009,Usina 1,0.567752,28,1,0,9,RB867515,73.9300,13.10,9.767632,LV21,Latossolos,Suave Ondulado,1,RB867515,1819,LV,1920,M1,25.337488,101.743631,729.276172,26.649752,89.265098,763.087132,25.242756,204.671192,538.555550,23.590764,127.704532,585.404973,0.000000,62.126239,62.126239,-0.168176,-0.044599,4.747651,4.441061,4.272886,3.762056,0.9,-4.061416,2.778855,-0.105161,61.390104,61.495265,0.062396,-0.478162,4.756425,4.107473,4.169868,0.559992,0.875,-4.156158,-0.583804,-0.054665,61.387866,61.442531,-0.256256,-0.058055,4.970627,4.142790,3.886534,-0.278974,0.777419,-4.098379,-1.057567,0.063435,62.666779,62.603344,0.874593,-0.021304,3.234026,1.085703,1.960296,0.623765,0.593333,-3.205408,-0.688433,-0.013128,62.523568,62.536696,-0.295239,-0.995675,2.314200,1.773951,1.478712,0.000000,0.654839,-2.677188,-0.818867,-0.015533,63.284102,63.299635,0.737150,0.452560,1.726665,0.786894,1.524044,0.000000,0.885,-2.779815,-1.271393,0.081596,62.112701,62.031106,1.454109,0.587939,1.868614,0.601366,2.055475,0.0,1.1,-3.941182,-1.850885,-0.220497,61.013739,61.234236,1.486130,0.428814,2.658608,1.565510,3.051641,0.741275,1.141935,-5.251300,-1.741476,0.170560,58.745347,58.574786,0.631924,-0.930612,3.493294,3.560029,4.191953,-0.251799,1.

# Save data

In [ ]:
# Define the output path
path = os.path.join(out_path, f'08_all_safra_without_outliers_merged.csv')
path

'/content/drive/MyDrive/Colab Notebooks/01_SugarcaneYieldPrediction/02_usinas_all/08_merged_safra_all/08_all_safra_without_outliers_merged.csv'

In [ ]:
# Save results as table
merged_data.to_csv(path, encoding='utf-8', index=False)
print('Dowanloaded to: ', path)

Dowanloaded to:  /content/drive/MyDrive/Colab Notebooks/01_SugarcaneYieldPrediction/02_usinas_all/08_merged_safra_all/08_all_safra_without_outliers_merged.csv
